In [44]:
import pandas as pd
from pymongo import MongoClient
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', 500)
client = MongoClient ('localhost', 27017)
data = client['companies'].companies

In [82]:
meta = ['name', 'category_code', 'number_of_employees', 'founded_year', 'total_money_raised']

def get_locations(data):
    return json_normalize(data, record_path = 'offices', meta = meta)


def get_companies_df(data):
    #df = pd.DataFrame(data)
    locations = get_locations(data)
    return(locations)
    
    '''
    gelocs = locations.apply(lambda e: [e["coord"][0],e["coord"][1]], result_type="expand", axis=1)
    clean_df = pd.concat([bks["restaurant_id"],locations[["street","zipcode"]],gelocs], axis=1)
    clean_df.rename({1:"lat",0:"long"}, axis=1, inplace=True)
    return clean_df
    '''

In [83]:
#Buscamos empresas dedicadas al sector del software y video juegos principalmente
result = data.find({}, {
    "name": 1,
    "category_code": 1,
    "_id": 0
})

datasets = []
for i in result:
 datasets.append(pd.DataFrame(i, index = [0]))

sector_df = pd.concat(datasets)

print(sector_df['category_code'].unique())

['web' 'enterprise' 'software' 'news' 'social' 'network_hosting'
 'games_video' 'music' 'mobile' 'search' 'advertising' 'messaging'
 'security' 'photo_video' 'finance' 'hardware' 'ecommerce' 'travel'
 'public_relations' 'other' 'real_estate' 'semiconductor' 'analytics'
 'health' 'legal' 'sports' 'biotech' 'cleantech' 'education' 'consulting'
 'transportation' None 'hospitality' 'fashion' 'nonprofit' 'nanotech'
 'automotive' 'design' 'manufacturing' 'government' 'local' 'medical']


In [28]:
#Analizamos los diferentes sectores de las compañías para ver cúales pueden ser los más interesantes para buscar similares

'''
['web', 'software', 'social', 'network_hosting'
 'games_video', photo_video', 'mobile', 'search', 'ecommerce', 'consulting', 'nanotech']

'''


#Buscamos compañías relativamente modernos, fundadas por lo menos a partir del año 90 (para incluir a Google)
#'founded_year: {$gte: 1990}'

#'Buscamos que un importante ratio de las empresas tenga IPO'
#En caso de no tener IPO, buscar si tiene acqusition.price_amount.
#Incluir algunas empresas que no tienen ni IPO ni precio de acqusition, pero su fundación es de hace poco tiempo.



#Buscamos empresas dedicadas al sector del software y video juegos principalmente

 

"\n['web', 'software', 'social', 'network_hosting'\n 'games_video', photo_video', 'mobile', 'search', 'ecommerce', 'consulting', 'nanotech']\n\n"

In [84]:
#Buscamos todas aquellas que tienen IPO y que su fecha de fundación es posterior a 1990
have_ipo = data.find({'ipo': {'$exists': True, '$not': {'$eq': None}}, 'founded_year': {'$gte': 1990}})

display(have_ipo.count())

display((get_companies_df(have_ipo)))

/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


218

,address1,address2,city,country_code,description,latitude,longitude,state_code,zip_code,name,category_code,number_of_employees,founded_year,total_money_raised
0,1601 Willow Road,,Menlo Park,USA,Headquarters,37.416050,-122.151801,CA,94025,Facebook,social,5299,2004,$2.43B
1,,,Dublin,IRL,Europe HQ,53.344104,-6.267494,None,,Facebook,social,5299,2004,$2.43B
2,340 Madison Ave,,New York,USA,New York,40.755716,-73.979247,NY,10017,Facebook,social,5299,2004,$2.43B
3,1355 Market St.,,San Francisco,USA,,37.776805,-122.416924,CA,94103,Twitter,social,1300,2006,$1.16B
4,2145 Hamilton Avenue,,San Jose,USA,Headquarters,37.295005,-121.930035,CA,95125,eBay,web,15000,1995,$6.7M
5,701 First Avenue,,Sunnyvale,USA,None,37.418531,-122.025485,CA,94089,Yahoo!,web,13600,1994,$6.8M
6,"60 E. Rio Salado Pkwy, Suite 400",,Tempe,USA,,33.429859,-111.938739,AZ,85281,LifeLock,security,644,2005,$178M
7,1600 Amphitheatre Parkway,,Mountain View,USA,Google Headquarters,37.421972,-122.084143,CA,,Google,search,28000,1998,$555M
8,112 S. Main St.,2nd Floor,Ann Arbor,USA,Google Ann Arbor,42.280988,-83.748882,MI,48104,Google,search,28000,1998,$555M
9,10 10th Street NE,Suite 600,Atlanta,USA,Google Atlanta,33.781466,-84.387519,GA,30309,Google,search,28000,1998,$555M


In [57]:
#Filtramos por sectores y además por compañías que disponen de al menos una oficina con coordenadas válidas no nulas:

categories = ['web', 'software', 'social', 'network_hosting', 'games_video', 
              'photo_video', 'mobile', 'search', 'ecommerce', 'consulting', 'nanotech']

'''
valid_coordinates = data.find({'$and': [
    {'offices': {'$exists': True}}, 
    {'offices.0.latitude': {'$ne': None}}, 
    {'offices.0.longitude': {'$ne': None}},
    {'category_code': {'$in': categories}}
]})
'''
valid_coordinates = data.find({
    'offices': {'$exists': True, '$ne': []}, 
    'offices.latitude': {'$ne': None}, 
    'offices.longitude': {'$ne': None},
    'category_code': {'$in': categories},
    'ipo': {'$ne': None},
    'founded_year': {'$gte': 1990}
    })


display('valid_coordinates:', valid_coordinates.count())

for i in list(valid_coordinates):
    display(i['name'], i['offices'][0]['latitude'])



/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


'valid_coordinates:'

64

'Facebook'

37.41605

...

43.670233

'Mobile TeleSystems OJSC'

55.755786

'Sitestar'

37.362749

In [47]:
#Buscamos todas aquellas que no tienen IPO pero tienen price amount y su fecha de fundación es posterior a 1990
categories = ['web', 'software', 'social', 'network_hosting', 'games_video', 
              'photo_video', 'mobile', 'search', 'ecommerce', 'consulting', 'nanotech']

have_ipo = data.find({
    '''
    'offices': {'$exists': True}, 
    'offices.0.latitude': {'$ne': None}, 
    'offices.0.longitude': {'$ne': None},
    '''

    'category_code': {'$in': categories},
    'ipo': {'$ne': None},
    'founded_year': {'$gte': 1990}
})
                                   
display(have_ipo.count())                                

/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


0

In [59]:
#No tienen IPO pero sí tiene acqusition.price_amount y su fecha de fundación es posterior a 1990

have_ipo_or_acquisiton = data.find({
    'offices': {'$exists': True, '$ne': []},
    'offices.latitude': {'$ne': None}, 
    'offices.longitude': {'$ne': None},
    'category_code': {'$in': categories},
    '$or': [{'ipo': {'$ne': None}} , {'acquisition.price_amount': {'$ne': None}}],
    'founded_year': {'$gte': 1990}
})

display(have_ipo_or_acquisiton.count()) 

for i in list(have_ipo_or_acquisiton):
    print(i['number_of_employees'])

/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if sys.path[0] == '':


256

47
5299
None
1300
None
15000
50
13600
60
35
110
0
3
None
None
800
8
75
26
20
0
None
4800
28
68
None
10
None
5
600
None
None
None
None
None
None
None
800
None
70
None
264
550
300
40
None
None
None
None
None
None
0
30
None
None
3
None
0
None
None
None
None
None
None
None
500
None
350
28
60
None
None
8000
None
200
500
None
4400
76
10
2000
0
150
None
1250
18
None
None
None
None
None
80
66
14
None
10
None
None
None
800
85
None
None
29
275
None
380
210
None
70
260
None
None
85
60
300
None
10
None
None
None
None
None
120
30
None
None
2000
974
None
None
1400
35
600
None
500
None
13500
5
None
None
75
None
10
45
3
None
None
26
None
None
9
25
1500
62
5
300
45
None
None
150
23
150
None
18
None
210
None
None
18
None
210
None
None
None
None
None
None
225
None
None
None
5
None
12
None
None
9
None
None
100
None
300
50
400
160
92
354
650
None
None
None
None
None
None
175
None
None
350
None
None
144
None
None
175
1200
300
65
120
881
None
None
800
170
20
20
None
None
80
100
None
None
None
None
9
None
Non

In [60]:
ipos_df = json_normalize(have_ipo)
display(ipos_df)

""
